In [1]:
import numpy as np
import tensorflow as tf
import math
import os
from tensorflow.examples.tutorials.mnist import input_data as mnist_data

In [2]:
print ("Tensorflow version " + tf.__version__)

Tensorflow version 1.5.0


In [ ]:
#tf.set_random_seed(0)

In [3]:
mnist = mnist_data.read_data_sets("data", one_hot=True)#, reshape=True, validation_size=0)

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [4]:
def createPlaceHolders(n_width, n_height, n_depth, n_classes):
    X = tf.placeholder(tf.float32, [None,n_width, n_height, n_depth ])
    y = tf.placeholder(tf.float32,  [None, n_classes])
    step = tf.placeholder(tf.int32)
    return X,y,step

In [5]:
#X = tf.reshape(X_, shape = [-1, n_width, n_height, n_depth])

W1 = tf.get_variable("W1", shape=[5,5,1,K], initializer=tf.contrib.layers.xavier_initializer())
B1 = tf.Variable(tf.ones([K])/10)
W2 = tf.get_variable("W2", shape=[5,5,K,L], initializer=tf.contrib.layers.xavier_initializer())
B2 = tf.Variable(tf.ones([L])/10)
W3 = tf.get_variable("W3", shape=[4,4,L,M], initializer=tf.contrib.layers.xavier_initializer())
B3 = tf.Variable(tf.ones([M])/10)
W4 = tf.get_variable("W4", shape=[7,7,M,N], initializer=tf.contrib.layers.xavier_initializer())
B4 = tf.Variable(tf.ones([N])/10)
W5 = tf.get_variable("W5", shape=[N,10], initializer=tf.contrib.layers.xavier_initializer())
B5 = tf.Variable(tf.ones([class_size])/10)

In [22]:
def intialize_parameters(K,L,M,N):
    parameters = {}
    W1 = tf.get_variable('W1', shape = [5, 5, 1, K], initializer = tf.contrib.layers.xavier_initializer())
    
    #tf.Variable(tf.truncated_normal([5, 5, 1, K], stddev=0.1))  # 5x5 patch, 1 input channel, K output channels
    B1 = tf.Variable(tf.ones([K])/10)
    W2 = tf.get_variable('W2', shape=[5, 5, K, L], initializer = tf.contrib.layers.xavier_initializer())
    #tf.Variable(tf.truncated_normal([5, 5, K, L], stddev=0.1))
    B2 = tf.Variable(tf.ones([L])/10)
    W3 = tf.get_variable('W3', shape=[4, 4, L, M], initializer = tf.contrib.layers.xavier_initializer())
    #tf.Variable(tf.truncated_normal([4, 4, L, M], stddev=0.1))
    B3 = tf.Variable(tf.ones([M])/10)
    W4 = tf.get_variable('W4', shape=[7 * 7 * M, N], initializer = tf.contrib.layers.xavier_initializer())
    B4 = tf.Variable(tf.ones([N])/10)
    W5 = tf.get_variable('W5', shape=[N, 10], initializer = tf.contrib.layers.xavier_initializer())
    B5 = tf.Variable(tf.ones([10])/10)
    parameters = {"W1":W1, "B1":B1, "W2":W2, "B2":B2,"W3":W3,"B3":B3,"W4":W4, "B4":B4,"W5":W5, "B5":B5}
    return parameters

In [24]:
def predict(parameters,X, M, n_inputs, n_classes):
    #X,y = createPlaceHolders(n_inputs, n_classes)
    #parameters = intialize_parameter(K,L,M,N)
    s = 1 # stride
    conv1 = tf.nn.relu(tf.nn.conv2d(X, parameters['W1'], strides=[1,s,s,1], padding = 'SAME') + parameters['B1'])
    
    #max pooling
    pool1 = tf.nn.max_pool()

    s = 2 # stride
    y2 = tf.nn.relu(tf.nn.conv2d(y1, parameters['W2'], strides=[1,s,s,1], padding = 'SAME') + parameters['B2'])

    s = 2 # stride
    y3 = tf.nn.relu(tf.nn.conv2d(y2, parameters['W3'], strides=[1,s,s,1], padding = 'SAME') + parameters['B3'])

    #falttern
    yy = tf.reshape(y3,  [-1, 7 * 7 * M])  # shape =
    y4 = tf.nn.relu(tf.matmul(yy, parameters['W4']) + parameters['B4'])
    yLogit = tf.matmul(y4, parameters['W5']) + parameters['B5']

    
    
    return yLogit
    #accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [8]:
def find_loss(yLogit, y):
    entropy = tf.nn.softmax_cross_entropy_with_logits(logits=yLogit, labels=y)
    loss = tf.reduce_mean(entropy)
    return loss

In [28]:
def find_accuracy(yLogit, y):
    prediction = tf.nn.softmax(yLogit)
    correct_prediction = tf.equal(tf.argmax(prediction,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy

In [43]:
def model():
    
    mnist = mnist_data.read_data_sets("data", one_hot=True)
    
    n_classes = 10
    n_width   = 28
    n_height  = 28
    n_depth   = 1
    n_inputs = n_height * n_width * n_depth # total pixels
    
    # three convolution layers 
    K = 24   # First layer depth
    L = 48   # Second layer depth
    M = 64  # Third layer depth
    N = 300 # Fully connected 

    learning_rate = 0.001
    n_epochs = 10
    epochs = 550
    batch_size = 100
    n_batches = int(mnist.train.num_examples/batch_size)
    
    
    X, y, step = createPlaceHolders(n_width, n_height, n_depth, n_classes)
    parameters = intialize_parameters(K, L, M, N)
    yLogit     = predict(parameters, X, M, n_inputs, n_classes)
    loss       = find_loss(yLogit, y)
    accuracy   = find_accuracy(yLogit, y)
    
    #lr = 0.0001 +  tf.train.exponential_decay(0.003, step, 2000, 1/math.e)
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)
    
    
    acc = 0.0
    lo = 0.0
    
    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)
    for i in xrange(epochs):
        batch_X, batch_y = mnist.train.next_batch(batch_size)
        a, c, _ = sess.run([accuracy, loss , optimizer], feed_dict={X : batch_X.reshape([-1, n_width, n_height, n_depth]), y : batch_y, step : i})
        print(str(i) + ": accuracy:" + str(a) + " loss: " + str(c))
        acc = a + acc
        lo = c + lo
    print acc,lo
    print('Train accuracy is  ', acc/epochs)
    print('Train loss is  ', lo/epochs)
    acc = 0.0
    lo = 0.0
    for i in xrange(100):
        batch_X, batch_y = mnist.test.next_batch(batch_size)
        a, c = sess.run([accuracy, loss],
                                    feed_dict={X : batch_X.reshape([-1, n_width, n_height, n_depth]) , y : batch_y })
        print(str(i) + ": ********* epoch " + str(i*100//mnist.train.images.shape[0]+1) + " ********* test accuracy:" + str(a) + " test loss: " + str(c))
        acc = a + acc
        lo = c+lo
    print acc, lo
    print('Test accuracy is  ', acc/100)
    print('Test loss is  ', lo/100)
    sess.close()
    

In [44]:
tf.reset_default_graph() 
model()

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
0: accuracy:0.08 loss: 2.3071601
1: accuracy:0.13 loss: 2.5428808
2: accuracy:0.22 loss: 2.4272718
3: accuracy:0.21 loss: 2.355474
4: accuracy:0.14 loss: 2.243554
5: accuracy:0.45 loss: 2.2057447
6: accuracy:0.28 loss: 2.1616373
7: accuracy:0.22 loss: 2.2180433
8: accuracy:0.38 loss: 2.1213229
9: accuracy:0.61 loss: 1.99426
10: accuracy:0.51 loss: 1.9238063
11: accuracy:0.49 loss: 1.8067737
12: accuracy:0.49 loss: 1.741659
13: accuracy:0.75 loss: 1.4177612
14: accuracy:0.69 loss: 1.3690314
15: accuracy:0.73 loss: 1.143476
16: accuracy:0.72 loss: 0.9814473
17: accuracy:0.74 loss: 0.8174007
18: accuracy:0.77 loss: 0.857941
19: accuracy:0.76 loss: 0.7116574
20: accuracy:0.84 loss: 0.60502684
21: accuracy:0.76 loss: 0.5945478
22: accuracy:0.83 loss: 0.6836641
23: accuracy:0.8 loss: 0.59996134
24: accuracy:0.87 loss: 0.4511

228: accuracy:0.98 loss: 0.074018605
229: accuracy:0.93 loss: 0.23693833
230: accuracy:0.98 loss: 0.06617777
231: accuracy:0.96 loss: 0.18262435
232: accuracy:0.98 loss: 0.091337465
233: accuracy:0.97 loss: 0.07337758
234: accuracy:0.95 loss: 0.08443007
235: accuracy:0.97 loss: 0.16892937
236: accuracy:0.98 loss: 0.050849203
237: accuracy:0.97 loss: 0.090956725
238: accuracy:0.98 loss: 0.058062825
239: accuracy:0.94 loss: 0.2039483
240: accuracy:0.94 loss: 0.16756457
241: accuracy:0.98 loss: 0.077507354
242: accuracy:1.0 loss: 0.032516614
243: accuracy:0.95 loss: 0.105700165
244: accuracy:0.97 loss: 0.09734952
245: accuracy:0.98 loss: 0.068531305
246: accuracy:0.97 loss: 0.10094462
247: accuracy:0.99 loss: 0.035178937
248: accuracy:0.98 loss: 0.049404144
249: accuracy:0.98 loss: 0.07666638
250: accuracy:0.98 loss: 0.054590885
251: accuracy:0.96 loss: 0.11755136
252: accuracy:0.97 loss: 0.08513336
253: accuracy:0.98 loss: 0.07039977
254: accuracy:0.95 loss: 0.16076817
255: accuracy:0.98

456: accuracy:0.98 loss: 0.070658915
457: accuracy:0.98 loss: 0.04304279
458: accuracy:0.96 loss: 0.08991724
459: accuracy:0.97 loss: 0.10539034
460: accuracy:0.99 loss: 0.11231258
461: accuracy:0.98 loss: 0.09130039
462: accuracy:0.97 loss: 0.05756754
463: accuracy:0.94 loss: 0.11666539
464: accuracy:0.98 loss: 0.07792187
465: accuracy:0.98 loss: 0.108120434
466: accuracy:0.98 loss: 0.056289073
467: accuracy:0.97 loss: 0.09331726
468: accuracy:0.96 loss: 0.15416048
469: accuracy:1.0 loss: 0.014078429
470: accuracy:0.98 loss: 0.06416512
471: accuracy:1.0 loss: 0.026098799
472: accuracy:0.96 loss: 0.08235962
473: accuracy:0.96 loss: 0.10129404
474: accuracy:1.0 loss: 0.027519118
475: accuracy:0.98 loss: 0.060731735
476: accuracy:0.97 loss: 0.13697207
477: accuracy:0.97 loss: 0.12933873
478: accuracy:0.98 loss: 0.04646489
479: accuracy:1.0 loss: 0.022313466
480: accuracy:0.98 loss: 0.06173381
481: accuracy:0.98 loss: 0.035551134
482: accuracy:1.0 loss: 0.028654056
483: accuracy:0.98 loss

64: ********* epoch 1 ********* test accuracy:0.97 test loss: 0.10203644
65: ********* epoch 1 ********* test accuracy:0.99 test loss: 0.03019593
66: ********* epoch 1 ********* test accuracy:0.97 test loss: 0.07087984
67: ********* epoch 1 ********* test accuracy:1.0 test loss: 0.023167284
68: ********* epoch 1 ********* test accuracy:0.97 test loss: 0.10757292
69: ********* epoch 1 ********* test accuracy:1.0 test loss: 0.023797905
70: ********* epoch 1 ********* test accuracy:1.0 test loss: 0.02035376
71: ********* epoch 1 ********* test accuracy:1.0 test loss: 0.033162713
72: ********* epoch 1 ********* test accuracy:0.99 test loss: 0.020381512
73: ********* epoch 1 ********* test accuracy:0.97 test loss: 0.09308178
74: ********* epoch 1 ********* test accuracy:1.0 test loss: 0.015162871
75: ********* epoch 1 ********* test accuracy:0.97 test loss: 0.061512228
76: ********* epoch 1 ********* test accuracy:1.0 test loss: 0.014596667
77: ********* epoch 1 ********* test accuracy:0.98